In [35]:
import sqlalchemy as db
import getpass
import json
import pandas as pd
import requests
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/clara/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores("Oh shit")

{'neg': 0.783, 'neu': 0.217, 'pos': 0.0, 'compound': -0.5574}

In [21]:
password = getpass.getpass("Insert your mysql root password: ")
engine = db.create_engine('mysql+pymysql://root:{}@localhost/project_api'.format(password))
print("Connected to server!")

Insert your mysql root password: ········
Connected to server!


# return chats from iduser

In [65]:
data = requests.get('http://localhost:8080/idUser/10').json()
json.loads(data)

[{'idMessage': 63,
  'text': 'Oh mama',
  'datetime': '2019-12-01 21:28:21',
  'users_idUser': 31,
  'chats_idChat': 4}]

# return who is this idUser

In [68]:
whothe = requests.get('http://localhost:8080/mensajes/8').json()
whothe

'[{"userName":"John Wick","TOTAL_MESSAGES":12}]'

# return chat from idchat

In [61]:
chatitos = requests.get('http://localhost:8080/chat/4/list').json()
chatitos

[['REDRUM'],
 ['What was that?'],
 ['REDRUM REDRUM REDRUM'],
 ['Did you mean red room? Id love a red room.'],
 ['Murder, bro, cold blooded splashy murderous fun'],
 ['Chill out, pinky.. Its such a nice day'],
 ['A nice day to create havoc!'],
 ['Ill go for a walk in the park, I guess. Maybe fishing'],
 ['Fishing sounds good. What about the labyrinth?'],
 ['No! That places creeps me out..'],
 ['Oh mama']]

In [8]:
#https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-fetchall.html

str

In [50]:
def getSent(chats):
    analiza = ' '.join([word for frase in chats for word in frase])
    return sid.polarity_scores(analiza)

In [51]:
getSent(chatitos)

{'neg': 0.207, 'neu': 0.57, 'pos': 0.223, 'compound': -0.4261}

# create User

In [80]:
new_name = {'name': 'Felipe Ferrero'}
userid = requests.post("http://localhost:8080/user/create", data=new_name).json()
print(userid)

[[34], [35]]


# create Chat

In [67]:
def newChat():  
    with engine.connect() as conn:
        a=list(conn.execute("SELECT idChat FROM chats ORDER BY idChat DESC LIMIT 1"))
        new_chatId = a[0][0]+1
    with engine.connect() as conn:
        new_chatque ="""
            INSERT INTO chats (idChat) VALUES ({})  
        """.format(new_chatId)
        conn.execute(new_chatque) 
    return print("You've succesfully created {} chat".format(new_chatId))

In [68]:
newChat()

You've succesfully created 7 chat


In [90]:
chatid = requests.post("http://localhost:8080/create/chat").json()
print(chatid)

{'11': "You've succesfully created this chat"}


# create Message

In [93]:
new_message = {'text': 'Heyyyy, I love cats', 
               'userid': 3,
               'chatid': 10}
userid = requests.post("http://localhost:8080/chat/addmessage", data=new_message).json()
print(userid)

[[60], [61], [62], [66]]


# return Sentiments

In [62]:
sent = requests.get("http://localhost:8080/chat/4/sentiment").json()
print(sent)

{'neg': 0.207, 'neu': 0.57, 'pos': 0.223, 'compound': -0.4261}
